# 02467 Computational Social Science
## Assignment 1
### Group 15

Our GitHub repo is availabe at: https://github.com/Simo067m/ComSocSci-Assignments <br>
Contribution:
- s233304 : xxxx
- s214592 : xxxx

In [17]:
# Import packages
import pandas as pd
import networkx as nx
import netwulf
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import ast
import numpy as np

from multiprocessing import Pool
import time

## Part 1: Web-scraping
Web-scraping the list of participants to the International Conference in Computational Social Science

In [18]:
# Define a function for finding all unique researchers
def scrape_IC2S2(soup : BeautifulSoup):
    # Find all the names from the top table
    names = []
    # Find all the table rows
    table_rows = soup.find_all("tr")
    for tr in table_rows:
        tds = tr.find_all("td")
        for row in tds:
            a = row.find_all("a")
            for text in a:
                text_content = text.text
                if ("Keynote" in text_content):
                    text_split = text_content.split("-")
                    stripped = text_split[1].strip()
                    if (stripped not in names):
                        names.append(stripped)
    
    # Find all the names from the bottom lists
    # Find all the unordered lists
    ul = soup.find_all("ul", class_="nav_list")
    # Find all the list elements
    for list in ul:
        found_names = list.find_all("i")
        # For every found name line, seperate into individual names
        for name in found_names:
            found_names_seperated = name.text.split(", ")
            for seperated_name in found_names_seperated:
                if (seperated_name.strip() not in names):
                    names.append(seperated_name.strip())

    # Find all the names of the chairs
    headers = soup.find_all("h2")
    for header in headers:
        text = header.find("i")
        if (text is not None):
            seperated_name = text.text.split(": ")
            if (seperated_name[1].strip() not in names):
                names.append(seperated_name[1].strip())

    return names

In [19]:
# Define url and collect content
LINK = "https://ic2s2-2023.org/program"
r = requests.get(LINK)
soup = BeautifulSoup(r.content)

# find participant names
IC2S2_names = scrape_IC2S2(soup)
# Save to a pandas DataFrame
IC2S2_names_df = pd.DataFrame(IC2S2_names, columns=["name"])
IC2S2_names_df.to_csv("IC2S2_names.csv", index=False)

### Q5: How many unique researchers do you get?
>- 1491

In [20]:
print(len(pd.read_csv("IC2S2_names.csv")))

1491


### Q6: Explain the process you followed to web-scrape the page. Which choices did you make to accurately retreive as many names as possible? Which strategies did you use to assess the quality of your final list? Explain your reasoning and your choices **(answer in max 150 words)**.

>- By inspecting the webpage, we were able to figure out that names were always contained in an &lt;a&gt; element for displaying the name properly. This means that finding an &lt;a&gt; element within one of the tables containing schedules would guarantee a name. When finding other names, like the ones that have the "chair", correctness was ensured by splitting that part from the name, ensuring only the name is retrieved.
>- Before adding a new name to the list, there is a check making sure that the name is not already in the found names list before adding it, making sure only unique names are in the list. The names contain no unwanted whitespace by calling the $\texttt{str.strip()}$ method before adding.

## Part 2: Ready Made vs Custom Made Data

### Q1: What are pros and cons of the custom-made data used in Centola's experiment (the first study presented in the lecture) and the ready-made data used in Nicolaides's study (the second study presented in the lecture)? You can support your arguments based on the content of the lecture and the information you read in Chapter 2.3 of the book **(answer in max 150 words)**.

>- 

## Part 3: Gathering Research Articles using the OpenAlex API


In [21]:
BASE_URL = "https://api.openalex.org"
WORKS_URL = "/works"
CONCEPT_URL = "/concepts"

In [22]:
"""
Retrieve concepts ids and return tupel of lists that reflect ([Sociology, Psychology, Economics, Political Science], [Mathematics, Physics, Computer science])
"""

def create_concept_filters():
    #Retrieve concepts so they can be used for filtering
    response_concepts = requests.get(BASE_URL + CONCEPT_URL)
    all_concepts = (response_concepts.json()['results'])
    
    # We want either one of: Sociology OR Psychology OR Economics OR Political Science
    com_soc_sci_list = ['Sociology', 'Psychology', 'Economics', 'Political science']
    # AND intersecting with a quantitative discipline (Mathematics OR Physics OR Computer Science"
    quant_disc_list = ['Mathematics', 'Physics', 'Computer science']
           
    com_soc_sci_ids = [c['id'] for c in all_concepts if c['display_name'] in com_soc_sci_list]
    quant_disc_ids = [c['id'] for c in all_concepts if c['display_name'] in quant_disc_list]

    return(com_soc_sci_ids, quant_disc_ids)

In [23]:
"""
Returns a parallel executable "to-do" list of parameters for retrieving batches of works at once.
"""

def create_todo_list(author_id_list):
    todo_list = []
    batch_size = 25

    com_soc_sci_ids, quant_disc_ids = create_concept_filters()
    
    for i in range(0, len(author_id_list), batch_size):
        author_filter = '|'.join(author_id_list[i:i + batch_size])
        filter_string = f"authorships.author.id:{author_filter},cited_by_count:>10,authors_count:<10,concepts.id:{'|'.join(com_soc_sci_ids)},concepts.id:{'|'.join(quant_disc_ids)}"

        params = {
            "filter": filter_string,
            "per-page": 200,
            "cursor": "*"
        }
        todo_list.append(params)
    return todo_list

In [24]:
"""
Retrieve works for the given parameters using paging.
"""

def retrieve_all_works(params):
    
    # Pause for 2 seconds to stay within limits of API calls
    time.sleep(2)
    
    # List to store all works retrieved
    all_works = []
    
    # Make requests until all works are fetched
    while True:
        # Make the GET request
        response = requests.get(BASE_URL + WORKS_URL, params=params)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Append retrieved works to the list
            all_works.extend(response.json()["results"])
            
            # Check if there are more works to fetch (paging)
            if len(response.json()["results"]) == params["per-page"]:
                # Update the cursor for the next page
                params["cursor"] = response.json()["meta"]["next_cursor"]
            else:
                # No more works to fetch
                break
        else:
            # Print an error message if the request was not successful
            print("Error:", response.status_code, params)
            break
    
    return all_works

In [25]:
"""
Return object including id, publication_year, cited_by_count and author_ids.
"""

def create_paper(work):
    author_ids = [a['author']['id'] for a in work['authorships']] 
    return {
        'id': work['id'],
        'publication_year': work['publication_year'],
        'cited_by_count': work['cited_by_count'],
        'author_ids': author_ids
    }

In [26]:
"""
Return object including id, title and abstract_inverted_index.
"""

def create_abstract(work):
    return {
        'id': work['id'],
        'title': work['title'],
        'abstract_inverted_index': work['abstract_inverted_index']
    }

In [27]:
"""
Process retrieved data and extract relevant information.
"""
def process_data():
    for upper_list in unprocessed_data:
        for work in upper_list:
            IC2S2_papers.append(create_paper(work))
            IC2S2_abstracts.append(create_abstract(work))

### Retrieve and process data

In [28]:
IC2S2_papers = []
IC2S2_abstracts = []

papers_df = pd.DataFrame()
abstracts_df = pd.DataFrame()

In [29]:
authors = pd.read_csv('week2-authors.csv')

# Filters: Only include IC2S2 authors with a total work count between 5 and 5,000
filtered_authors = authors[(authors['works_count'] > 5) & (authors['works_count'] < 5000)]

In [30]:
"""if __name__ == "__main__":
    
    # Prepare all request parameters that have to be done in parallel
    todo_list = create_todo_list(filtered_authors.id.tolist())
    
    # Perform all requests with the prepared parameters in parallel
    with Pool() as p:
        unprocessed_data = list(tqdm(p.imap(retrieve_all_works, todo_list), total=len(todo_list)))

    # Process collected data
    process_data()   

    # Remove duplicates and store in dataframes and 
    papers_df = pd.DataFrame(IC2S2_papers).drop_duplicates(subset=['id'])
    abstracts_df = pd.DataFrame(IC2S2_abstracts).drop_duplicates(subset=['id'])
    
    # Export data
    papers_df.to_csv('papers.csv', index=False)
    abstracts_df.to_csv('abstracts.csv', index=False)"""

'if __name__ == "__main__":\n    \n    # Prepare all request parameters that have to be done in parallel\n    todo_list = create_todo_list(filtered_authors.id.tolist())\n    \n    # Perform all requests with the prepared parameters in parallel\n    with Pool() as p:\n        unprocessed_data = list(tqdm(p.imap(retrieve_all_works, todo_list), total=len(todo_list)))\n\n    # Process collected data\n    process_data()   \n\n    # Remove duplicates and store in dataframes and \n    papers_df = pd.DataFrame(IC2S2_papers).drop_duplicates(subset=[\'id\'])\n    abstracts_df = pd.DataFrame(IC2S2_abstracts).drop_duplicates(subset=[\'id\'])\n    \n    # Export data\n    papers_df.to_csv(\'papers.csv\', index=False)\n    abstracts_df.to_csv(\'abstracts.csv\', index=False)'

In [31]:
#Remove
papers_df = pd.read_csv("papers.csv")
abstracts_df = pd.read_csv("abstracts.csv")

authors_dict = {'id': papers_df['author_ids'].explode().tolist()}
authors_df = pd.DataFrame(authors_dict).drop_duplicates(subset=['id'])

### Q1 - Dataset summary
- _How many works are listed in your IC2S2 papers dataframe?_ 10.145
- _How many unique researchers have co-authored these works?_ 15.741

In [32]:
Q1_works = papers_df.shape[0]
print(f"How many works are listed in your IC2S2 papers dataframe? {Q1_works}")

How many works are listed in your IC2S2 papers dataframe? 10155


In [33]:
Q1_authors = len(set(papers_df['author_ids'].explode().tolist()))
print(f"How many unique researchers have co-authored these works? {Q1_authors}")

How many unique researchers have co-authored these works? 8766


### Q2 - Efficiency in code
_Describe the strategies you implemented to make your code more efficient. How did your approach affect your code's execution time?_

I have used multiprocessing of batches with paging while using filters when retrieving the data. 
First of all, I have filtered the authors from week 2 exercise 2 to start with a smaller set already. I have then created a list of requests that can be executed in parallel. Each of these requests includes the filters (described in the assignment) of the individual request, and the batch size of 200.
Next, I executed these requests in parallel using the function “imap” by the class “multiprocessing.Pool”. To keep within the limits of the API I have added a sleep for two seconds. 
The data retrieval has taken a total of about 36 seconds. (112 words)

### Q3 - Filtering Criteria and Dataset Relevance
_Reflect on the rationale behind setting specific thresholds for the total number of works by an author, the citation count, the number of authors per work, and the relevance of works to specific fields. How do these filtering criteria contribute to the relevance of the dataset you compiled? Do you believe any aspects of Computational Social Science research might be underrepresented or overrepresented as a result of these choices?_

When focusing on the total number of works by an author and the citation count, one can say, that while this filtering helps include work from established researchers with a proven track record of impactful research, it excludes upcoming researchers or those from underrepresented backgrounds or smaller fields of computational social science which might be as valuable as the others.
This could lead to an overrepresentation of researchers in a more general field and an underrepresentation of niche researchers.
When looking into the number of authors per work, can help to create a balanced representation of single-authored and multi-authored works. However, collaborative research projects which may be in interdisciplinary environments will be excluded.
When looking into the specific fields, research that does not directly fall into one of these pre-defined fields might still hold a lot of value within these fields and may bridge traditional disciplinary boundaries. (147 words)

## Part 4: The Network of Computational Social Scientists

### Constructing the Computational Social Scientists Network

### Part 4.1: Network Construction

In [34]:
# Load the papers dataset
papers_df = pd.read_csv("papers.csv")
# Load the author's dataset
authors_df = pd.read_csv("week2-authors.csv")
# Load the abstracts dataset
abstracts_df = pd.read_csv("abstracts.csv")

In [35]:
from itertools import combinations
from collections import defaultdict


# Create a dictionary to store the co-authorship counts
author_pairs = defaultdict(int)

# Iterate over each row in the DataFrame
for authors in tqdm(papers_df['author_ids'], desc="Authors"):
    authors = ast.literal_eval(authors)
    # Generate combinations of author IDs
    for pair in combinations(authors, 2):
        # Sort the pair to ensure consistent counting
        sorted_pair = tuple(sorted(pair))
        # Increment the co-authorship count for this pair
        author_pairs[sorted_pair] += 1

# Convert the dictionary to a list of tuples
weighted_edgelist = [(pair[0], pair[1], count) for pair, count in author_pairs.items()]

Authors: 100%|██████████| 10155/10155 [00:00<00:00, 47011.23it/s]


In [36]:
# Create weighted edgelist WIP
def create_weighted_edgelist(paper_authors_ids):
    edges = []
    pairs = []
    # Loop through the paper authors
    for authors in tqdm(paper_authors_ids, desc="Progress"):
        for i in range(len(authors)):
            for j in range(i + 1, len(authors)):
                pair = (authors[i], authors[j])
                if pair not in pairs and (pair[1], pair[0]) not in pairs:
                    co_authored_works_count = 0
                    pairs.append(pair)
                    for authors_ in paper_authors_ids:
                        if pair[0] in authors_ and pair[1] in authors_:
                            co_authored_works_count += 1
                    edges.append((pair[0], pair[1], co_authored_works_count))
    
    return edges

papers_authors_ids = papers_df["author_ids"].apply(ast.literal_eval)

#weighted_edgelist = create_weighted_edgelist(papers_authors_ids)

In [37]:
#edgelist = np.array(weighted_edgelist)
#np.save("edgelist.npy", edgelist)
#weighted_edgelist = np.load("weighted_edgelist_np.npy")

In [38]:
# Create graph
G = nx.Graph()
G.add_weighted_edges_from(weighted_edgelist)
print(G.number_of_nodes())

15746


In [41]:
authors_df = pd.read_csv("authors.csv")
papers_df = pd.read_csv("papers.csv")

"""# Convert authors_df to a dictionary for faster lookups
authors_dict = authors_df.set_index('id').to_dict(orient='index')

# Group papers_df by author_id for faster lookup
papers_grouped = papers_df.groupby('author_ids')

for node in tqdm(G.nodes, desc="Nodes"):
    remove_list = []
    author_id = node
    display_name = ""
    country_code = ""
    # Find the author's name and country code from the authors dataset
    for _, author in authors_df.iterrows():
        if author["id"] == author_id:
            display_name = author["display_name"]
            country_code = author["country_code"]
    
    # Find the author's citation count and first publication from the papers dataset
    papers_publication_years = []
    citation_count = 0
    for _, paper in papers_df.iterrows():
        if author_id in paper["author_ids"]:
            papers_publication_years.append(paper["publication_year"])
            citation_count += paper["cited_by_count"]
    try:
        year_of_first_publication = min(papers_publication_years)
    except:
        year_of_first_publication = 0

    G.nodes[node]["display name"] = display_name
    G.nodes[node]["country"] = country_code
    G.nodes[node]["citation count"] = citation_count
    G.nodes[node]["first publication year"] = year_of_first_publication"""

'# Convert authors_df to a dictionary for faster lookups\nauthors_dict = authors_df.set_index(\'id\').to_dict(orient=\'index\')\n\n# Group papers_df by author_id for faster lookup\npapers_grouped = papers_df.groupby(\'author_ids\')\n\nfor node in tqdm(G.nodes, desc="Nodes"):\n    remove_list = []\n    author_id = node\n    display_name = ""\n    country_code = ""\n    # Find the author\'s name and country code from the authors dataset\n    for _, author in authors_df.iterrows():\n        if author["id"] == author_id:\n            display_name = author["display_name"]\n            country_code = author["country_code"]\n    \n    # Find the author\'s citation count and first publication from the papers dataset\n    papers_publication_years = []\n    citation_count = 0\n    for _, paper in papers_df.iterrows():\n        if author_id in paper["author_ids"]:\n            papers_publication_years.append(paper["publication_year"])\n            citation_count += paper["cited_by_count"]\n    tr

In [51]:
# Convert authors_df to a dictionary for faster lookups
authors_dict = authors_df.set_index('id').to_dict(orient='index')

# Group papers_df by author_id for faster lookup
papers_grouped = papers_df.groupby('author_ids')

for node in tqdm(G.nodes, desc="Nodes"):
    author_id = node
    display_name = ""
    country_code = ""
    
    # Find the author's name and country code from the authors_dict
    author_info = authors_dict.get(author_id)
    if author_info:
        display_name = author_info.get("display_name", "")
        country_code = author_info.get("country_code", "")

    # Find the author's citation count and first publication from the papers_grouped
    papers_publication_years = []
    citation_count = 0

    print(papers_grouped.groups)

    #if author_id in papers_grouped.groups:
    papers_subset = papers_grouped.get_group(author_id)
    papers_publication_years = papers_subset["publication_year"].tolist()
    citation_count = papers_subset["cited_by_count"].sum()
    year_of_first_publication = min(papers_publication_years)

    G.nodes[node]["display name"] = display_name
    G.nodes[node]["country"] = country_code
    G.nodes[node]["citation count"] = citation_count
    G.nodes[node]["first publication year"] = year_of_first_publication

Nodes:   0%|          | 0/15746 [00:00<?, ?it/s]

{'['https://openalex.org/A5000017075', 'https://openalex.org/A5020270223', 'https://openalex.org/A5057650206']': [5120], '['https://openalex.org/A5000046482', 'https://openalex.org/A5076755077']': [9178], '['https://openalex.org/A5000077728', 'https://openalex.org/A5029945844']': [4954], '['https://openalex.org/A5000092386', 'https://openalex.org/A5044459771', 'https://openalex.org/A5029882049']': [8916], '['https://openalex.org/A5000092386', 'https://openalex.org/A5070290348', 'https://openalex.org/A5029882049']': [8944], '['https://openalex.org/A5000093816', 'https://openalex.org/A5006921976', 'https://openalex.org/A5012331097', 'https://openalex.org/A5086924889', 'https://openalex.org/A5065912071', 'https://openalex.org/A5026412812', 'https://openalex.org/A5068384426']': [2283], '['https://openalex.org/A5000093816', 'https://openalex.org/A5010915106', 'https://openalex.org/A5086574543', 'https://openalex.org/A5005224649']': [2264], '['https://openalex.org/A5000138623', 'https://open

KeyError: 'https://openalex.org/A5046943453'

In [69]:
# Assuming G, authors_df, and papers_df are already defined

# Convert authors_df to a dictionary for faster lookups
authors_dict = authors_df.set_index('id').to_dict(orient='index')

# Group papers_df by author_ids for faster lookup
papers_df["author_ids"] = papers_df['author_ids'].apply(tuple)  # Convert lists to tuples
papers_grouped = papers_df.groupby('author_ids')

for node in tqdm(G.nodes, desc="Nodes"):
    author_id = node
    display_name = ""
    country_code = ""
    
    # Find the author's name and country code from the authors_dict
    author_info = authors_dict.get(author_id, {})
    display_name = author_info.get("display_name", "")
    country_code = author_info.get("country_code", "")

    # Find the author's citation count and first publication from the papers_grouped
    papers_publication_years = []
    citation_count = 0
    print(papers_df["author_ids"])
    # Iterate over the lists in "author_ids"
    if any(author_id in author_ids for author_ids in papers_df["author_ids"]):
        papers_subset = papers_grouped.get_group(tuple(author_id))
        papers_publication_years = papers_subset["publication_year"].tolist()
        citation_count = papers_subset["cited_by_count"].sum()

    year_of_first_publication = min(papers_publication_years, default=0)

    G.nodes[node]["display name"] = display_name
    G.nodes[node]["country"] = country_code
    G.nodes[node]["citation count"] = citation_count
    G.nodes[node]["first publication year"] = year_of_first_publication

Nodes:   0%|          | 12/15746 [00:00<04:32, 57.69it/s]

0        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
1        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
2        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
3        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
4        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
                               ...                        
10150    ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
10151    ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
10152    ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
10153    ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
10154    ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
Name: author_ids, Length: 10155, dtype: object
0        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
1        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
2        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
3        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
4        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
         

Nodes:   0%|          | 31/15746 [00:00<04:17, 61.11it/s]

0        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
1        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
2        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
3        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
4        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
                               ...                        
10150    ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
10151    ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
10152    ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
10153    ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
10154    ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
Name: author_ids, Length: 10155, dtype: object
0        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
1        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
2        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
3        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
4        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
         

Nodes:   0%|          | 38/15746 [00:00<04:27, 58.63it/s]

...

Nodes:   2%|▏         | 309/15746 [00:05<04:54, 52.50it/s]

0        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
1        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
2        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
3        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
4        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
                               ...                        
10150    ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
10151    ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
10152    ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
10153    ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
10154    ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
Name: author_ids, Length: 10155, dtype: object
0        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
1        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
2        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
3        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
4        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
         

Nodes:   2%|▏         | 310/15746 [00:05<04:48, 53.43it/s]

0        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
1        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
2        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
3        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
4        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
                               ...                        
10150    ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
10151    ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
10152    ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
10153    ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
10154    ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
Name: author_ids, Length: 10155, dtype: object
0        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
1        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
2        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
3        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
4        ([, ', h, t, t, p, s, :, /, /, o, p, e, n, a, ...
         

KeyboardInterrupt: 

In [64]:
import json
graph_json = nx.json_graph.node_link_data(G)

# Save to JSON
with open("network.json", "w") as json_file:
    json.dump(graph_json, json_file)